# 2019 빅콘테스트 퓨처스리그

## 1. 데이터 이해 및 EDA

### 1.1 데이터 통일

* Train Data와 Test Data의 변수가 다르므로 REG, ATT, IRR, DRR, CNL, CNR 변수를 제거하여 Test Data의 변수에 맞게 통일시킨다.

* 월별 지연율이 상이하다는 것과 A/C지연을 제외하면 날씨관련 지연이 가장 많다는 것을 확인하여 우리나라의 경우 기간에 따라 날씨의 특성이 다른점을 이용해 Test Data와 동일한 기간을 이용하여 예측하기로 결정했다.  
즉, 2017/09/16 ~ 2017/09/30, 2018/09/16 ~ 2018/09/30 데이터를 이용해 Test Data 예측

### 1.2 데이터셋 구분

Train Data와 Test Data에 대한 데이터 범위를 확인하던 중 Test Data에서만 나타나는 FLO, FLT가 발견되어 이후 모델링 과정에서 각각 다른 모델을 적용하기 위해 분리함.  
##### Detailed Dataset
1) FLO == M 인 Test Data(이후 모델링에서 FLT, FLO 관련 변수 제외)  
2) Test.FLT.unique not in Train.FLT.unique 인 Test Data(이후 모델링에서 FLT 관련 변수 제외)  
3) 위 2개를 제외한 기본적인 Test Data

### 1.3 데이터 전처리 및 EDA

시간관련 변수

* 변수생성 : SDT_YY, SDT_MM, SDT_DD, STT 변수를 이용해 YMD 변수(datetime) 생성 이후 YMD 변수를 이용하여 hour, minute 변수를 추가적으로 생성
* 데이터 범위 동일 : Train Data에만 존재하는 hour(23, 0)가 있어 Test Data 범위에 맞게 Train Data 삭제
* 시간대별 지연율 : Train.hour을 groupby를 이용해 시간대별 지연율을 계산하였더니 오후 시간대와 오전 시간대에 따라 지연율 양상이 다른 것을 확인하고 변수로 추가 채택

경로관련 변수
* 변수생성 : ARP, ODP 변수를 이용해 "경로" 라는 변수 생성 ex) ARP3에서 ARP6으로 가는 경우 3_6으로 제작
* 외부 데이터 활용 : 한국공항공사의 공항별 통계를 이용해 ARP, ODP을 각 공항에 대치
* ARP, ODP, 경로별 지연율 : ARP, ODP별 지연율을 계산했을 때, 공항에 따른 지연율 차이가 있었지만 지연율에 따른 공항의 특징을 확인하기 어려웠으나 이후 경로 변수를 만들고 경로별 지연율을 계산했을 때, 경로에 ARP3(제주)이 포함되는 경우 높은 지연율을 보인다는 특징을 확인할 수 있어 변수로 추가 채택

## 2. 외부데이터 활용

### 2.1 시계열 분석(LSTM)을 활용한 날씨 예측

* 날씨 예측 이유 : 항공기 지연의 대부분인 A/C지연을 제외하면 최초지연의 경우, 날씨관련 지연이 가장 많다.
* 시계열 분석 사용 이유 : 날씨는 연속적으로 측정하는 변수이며, 분석 대상으로 설정한 데이터 또한 일정한 시간 간격으로 구성되어 있었기 때문
* LSTM 사용한 이유 : LSTM은 순환 신경망으로 은닉층의 상태를 업데이트할 때, 시계열 데이터에서의 패턴을 감지할 수 있기 때문이다.
* 분석 프로세스 : 각 공항의 시간별 기온, 강수량, 전운량, 풍속 확인 후 특정한 패턴을 확인할 수 없었던 기온, 강수량, 풍속을 제외하고 일별 전운량을 통해 Test Data 기간의 일별 전운량 예측

## 3. 내부데이터 변수 생성

### 3.1 Groupby 변수 생성

* Train Data의 범주형 변수 : SDT_DY, ARP, ODP, FLO, FLT, AOD(Train Data의 대부분)
* Groupby 변수 생성 : 범주형 변수를 일반적으로 사용하는 원-핫 인코딩을 이용하여 변환하였을 때, 너무 많은 변수가 생겨 차원의 저주가 발생하게 되는데 이를 방지하기 위해 해당 범주의 수치적 특성을 이용해 대치하는 방식을 채택, 추가적으로 요일별 지연율에서는 한 주의 주중/주말, 주초/주후에 따라 지연율 차이를 보이고 항공사별 지연율에서는 대형/소형 항공사에 따라 지연율 차이를 보여 이에 대해서 변수 추가 생성  
(요일별지연율, 출발/도착공항별 지연율, 항공사별지연율, 경로별지연율, 편명별지연율, 공항별 주중주말 지연비율, 시간대별 지연건수, 출발/도착공항 초반후반 지연율, 항공사크기별 지연율)

## 4. 모델링

### 4.1 Basic Model

LogisticRegression, DecisionTreeClassifier, RandomForestClassifier, KNeighborsClassifier, MLPClassifier, XGBClassifier, GradientBoostingClassifier, LGBMClassifier 사용  
* Model 선택 : Cross_Validataion을 진행하여 성능을 확인  
LogisticRegression, MLPClassifier, XGBClassifier, GradientBoostingClassifier, LGBMClassifier Model만을 사용하기로 결정

### 4.2 Imbalance Learning

Imbalance Learing : Train Data에서의 약 90%가 DLY == N 이라는 점에서 데이터가 편향되어 있다는 것을 알 수 있었고, 편향된 데이터에서 더 많은 값인 DLY == N 을 줄이거나 DLY == Y 값을 임의로 생성하여 편향을 줄이는 방식인 Imbalance Learning 진행  
##### Detailed method   

1) TomekLink : 마이너값(Y)과 가장 가까이에 있는 메이저값(N)을 제거  
2) EasyEnsemble : Random Undersampling을 반복적으로 적용하여 Ensemble Set를 구축하는 방식

### 4.3 Parameter Tuning

* Model에 대한 현재의 Data를 가장 알맞게 학습할 수 있는 Parameter를 탐색

### 4.4 Ensemble

* 5개의 Model을 Voting 기법을 통해 앙상블 진행